In [34]:
import tqdm as tqdm
from keras.models import *
from keras.layers import *
from keras.layers.convolutional import *
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.python.keras.utils.np_utils import to_categorical
from torchvision.transforms import transforms
import math

### Collab code!

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

In [ ]:
!kaggle datasets download -d mostafaabla/garbage-classification -p /content/sample_data/ --unzip

In [ ]:
#data processing
def read_and_resize(filename, grayscale = False, fx= 1, fy=1):
    if grayscale:
        img_result = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    else:
        imgbgr = cv2.imread(filename, cv2.IMREAD_COLOR)
        img_result = cv2.cvtColor(imgbgr, cv2.COLOR_BGR2RGB)
    img_result = cv2.resize(img_result, None, fx=fx, fy=fy, interpolation = cv2.INTER_CUBIC)
    return img_result

#show images
def show_in_row(list_of_images, titles = None, disable_ticks = False):
    count = len(list_of_images)
    for idx in range(count):
        subplot = plt.subplot(1, count, idx+1)
        if titles is not None:
            subplot.set_title(titles[idx])

        img = list_of_images[idx]
        cmap = 'gray' if (len(img.shape) == 2 or img.shape[2] == 1) else None
        subplot.imshow(img, cmap=cmap)
        if disable_ticks:
            plt.xticks([]), plt.yticks([])
    plt.show()

#open txt file using path to the file and file_name.txt
def openWithTxt(txt: str, path:str):
    outData = []
    with open("/content/sample_data/"+txt) as f:
        for i in f.readlines():
            img ,classNum = i.split()
            classNum = int(classNum)
            img = read_and_resize(path+f"{deIdx[classNum]}/"+img).astype("float32")/255
            s = img.shape
            img = cv2.resize(img, (s[0],s[0]), interpolation = cv2.INTER_AREA)
            #outData.extend(rotate2sides(img, classNum))
            outData.append((img, classNum))
    return np.array(outData)

def openWith(dfX, dfY):
    outData = []
    for i in range(len(dfX)):
        path = "./sample_data/garbageAll/"
        img = dfX[i]
        classNum = dfY[i]
        img = read_and_resize(path+img).astype("float32")/255
        img = cv2.resize(img, (200,200), interpolation = cv2.INTER_AREA)
        outData.append((img, classNum))
    return np.array(outData)

# takes image ans its classNumber, and produce 4 rotaded images on exact degree (0, 90, 180, 270)
def rotate4sides(img, classNum):
    outData = []
    outData.append(np.array([img, classNum]))
    outData.append(np.array([cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE), classNum]))
    outData.append(np.array([cv2.rotate(img, cv2.ROTATE_180), classNum]))
    outData.append(np.array([cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE), classNum]))
    return outData

def rotate2sides(img, classNum):
    outData = []
    outData.append(np.array([img, classNum]))
    outData.append(np.array([cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE), classNum]))
    return outData

In [ ]:
import os
import shutil

def moveToAll(neededDirs):
    pathTo   = "./sample_data/garbageAll/"
    pathFrom = "./sample_data/garbage_classification/"

    if not os.path.isdir(pathTo):
        os.mkdir(pathTo)
    else:
        shutil.rmtree(pathTo)
        os.mkdir(pathTo)

    for folder in neededDirs:
        print(len(os.listdir(pathFrom+folder)))
        for filename in os.listdir(pathFrom+folder):
            shutil.copy2(pathFrom+folder+"/"+filename, pathTo)

neededDirs = ["brown-glass", "cardboard", "green-glass", "plastic", "paper", "white-glass", "metal"]
moveToAll(neededDirs)

#upload .csv in a folder


In [ ]:
files.upload()

In [ ]:
import pandas as pd
df = pd.read_csv("preprocessed_data.csv")
df

In [ ]:
from sklearn.model_selection import train_test_split

classDecode = {1:"paper/cardboard", 2: "metal", 3: "plastic", 4: "glass"}

x, x_test, y, y_test = train_test_split(df["photo"],df["class"],test_size=0.2,train_size=0.8)
x_train, x_cv, y_train, y_cv = train_test_split(x,y,test_size = 0.20,train_size =0.80)


In [ ]:
x_test.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)
x_train.reset_index(inplace=True, drop=True)
x_cv.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_cv.reset_index(inplace=True, drop=True)

In [ ]:
y_test

In [ ]:
testData = openWith(x_test, y_test)
xTest = np.stack(testData[:,0])
yTest =  to_categorical(testData[:,1],num_classes = len(classDecode.values())+1)[:,1:]
testData.shape

In [ ]:
trainData = openWith(x_train, y_train)
xTrain = np.stack(trainData[:,0])
yTrain = to_categorical(trainData[:,1],num_classes = len(classDecode.values())+1)[:,1:]
trainData.shape

In [ ]:
valData = openWith(x_cv, y_cv)
xVal = np.stack(valData[:,0])
yVal = to_categorical(valData[:,1],num_classes = len(classDecode.values())+1)[:,1:]
valData.shape

In [ ]:
yTrain.shape

In [ ]:
for i in range(len(x_test)):
    path = "./sample_data/garbageAll/"
    img = x_test[i]
    img = read_and_resize(path+img).astype("float32")/255
    img = cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)
    show_in_row([img])
    break

        

In [ ]:
show_in_row(xTest[:5], yTest[:5])

# Model 1

##  CNN (keras)


# model

# model pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import torch.optim as optim
import torchvision.models as models

In [ ]:
#output - vector of vectors, labels - vactor
def accuracy(outputs, labels):
    return torch.tensor(torch.sum(outputs.argmax(1) == labels.argmax(1)).item() / len(labels))

#implement Base for Classification
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        #print(batch.shape)
        #print()
        #batch - (img, label)
        self.train() 
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        #batch - (img, label)
        self.eval() 
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        

    def validation_epoch_end(self, outputs):
        #outputs - list of dicts of validations 
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print(f"Epoch {epoch+1}: train_loss: {result['train_loss']}, val_loss: {result['val_loss']}, val_acc: {result['val_acc']}")

class ImageClassificationInception(nn.Module):
    def training_step(self, batch):
        #print(batch.shape)
        #print()
        #batch - (img, label)
        self.train() 
        images, labels = batch 
        outputs = self(images) 
        loss1 = F.cross_entropy(outputs, labels)
        loss = loss1
        return loss
    
    def validation_step(self, batch):
        #batch - (img, label)
        self.eval() 
        images, labels = batch 
        outputs = self(images) 
        loss1 = F.cross_entropy(outputs, labels)
        loss = loss1
        acc = accuracy(outputs, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        

    def validation_epoch_end(self, outputs):
        #outputs - list of dicts of validations 
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print(f"Epoch {epoch+1}: train_loss: {result['train_loss']}, val_loss: {result['val_loss']}, val_acc: {result['val_acc']}")

In [ ]:

class ResNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet50(pretrained=True)
        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs, len(classDecode.values()))
    
    def forward(self, xb):
        o = self.network(xb)
        return F.softmax(o)

class DenseNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = models.densenet121(pretrained=True)
        self.network.classifier = nn.Linear(1024, len(classDecode.values()))
    
    def forward(self, xb):
        o = self.network(xb)
        return F.softmax(o)

class InceptionNet(ImageClassificationInception):
    def __init__(self):
        super().__init__()
        self.network = models.inception_v3(pretrained=True)
        self.network.aux_logits=False
        num_ftrs = self.network.AuxLogits.fc.in_features
        self.network.AuxLogits.fc = nn.Linear(num_ftrs, len(classDecode.values()))
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs,len(classDecode.values()))
    
    def forward(self, xb):
        o = self.network(xb)
        return F.softmax(o)


class VGGNet(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.vgg16(pretrained=True)
        # Replace last layer
        self.network.classifier[6] = torch.nn.Linear(4096, len(classDecode.values()))
    
    def forward(self, xb):
        o = self.network(xb)
        return F.softmax(o)

class Net(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=(1,1))
        self.conv2 = nn.Conv2d(16, 16, 3, padding=(1,1))
        self.conv3 = nn.Conv2d(16, 32, 3, padding=(1,1))
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv5 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv6 = nn.Conv2d(64, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.2)
        self.fc1 = nn.Linear(32 * 4 * 4 , 128)
        self.fc3 = nn.Linear(128, 6)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))
        x = self.pool(F.relu(self.conv6(x)))
        #print(x.shape)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc3(x)
        return F.softmax(x)


In [ ]:
torch.cuda.empty_cache()

In [ ]:
from torch.utils.data import Dataset


def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, x, device, len):
        self.dl = x
        self.device = device
        self.len = len
        
    def __iter__(self):
        for item in self.dl: 
            yield to_device(item, self.device)

    def __len__(self):
        return self.len

class MyData(Dataset):
    def __init__(self, x, y, transform):
        self.x = x
        self.y = y
        self.transform = transform
        self.len = len(x)
        self.to_tensor = transforms.ToTensor()
    def __len__(self):
        return self.len
    def __getitem__(self, item):
        #print(type(self.x[item]))
        return self.transform(self.x[item]), torch.from_numpy(self.y[item])

device = get_default_device()
device

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
batch_size = 32

print(xTrain.shape)
print(yTrain.shape)


myDataTrain = MyData(xTrain,yTrain, transform)
myDataVal = MyData(xVal,yVal, transform)
myDataTest = MyData(xTest,yTest, transform)

dataTrain = torch.utils.data.DataLoader(myDataTrain, batch_size=batch_size)
dataVal = torch.utils.data.DataLoader(myDataVal, batch_size=batch_size)
dataTest = torch.utils.data.DataLoader(myDataTest, batch_size=batch_size)

dataTrain = DeviceDataLoader(dataTrain, device,len(yTrain))
dataVal = DeviceDataLoader(dataVal, device,len(yVal))
dataTest = DeviceDataLoader(dataTest, device,len(yTest))



In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

# Inception

In [ ]:
model = to_device(InceptionNet(), device)
evaluate(model, dataTest)

In [ ]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 5.5e-5

history = fit(num_epochs, lr, model, dataTrain, dataVal, opt_func)

In [ ]:
evaluate(model, dataTest)

In [ ]:
del(model)

# DenseNet


In [ ]:
model = to_device(DenseNet(), device)
evaluate(model, dataTest)

In [ ]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 5.5e-5 

history = fit(num_epochs, lr, model, dataTrain, dataVal, opt_func)

In [ ]:
evaluate(model, dataTest)

In [ ]:
del(model)

# ResNet

In [ ]:
model = to_device(ResNet(), device)
evaluate(model, dataTest)

In [ ]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 5.5e-5 

history = fit(num_epochs, lr, model, dataTrain, dataVal, opt_func)

In [ ]:
evaluate(model, dataTest)

In [ ]:
del(model)

# VGG


In [ ]:
model3 = to_device(VGGNet(), device)
evaluate(model3, dataTest)

In [ ]:
num_epochs = 15
opt_func = torch.optim.Adam
lr = 5.5e-5 

history = fit(num_epochs, lr, model3, dataTrain, dataVal, opt_func)

In [ ]:
evaluate(model3, dataTest)

In [ ]:
del(model3)

In [57]:
del(model)

# ResNet

In [61]:
model = to_device(ResNet(), device)
evaluate(model, dataTest)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


{'val_loss': 1.3815927505493164, 'val_acc': 0.23726190626621246}

In [62]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 5.5e-5 

history = fit(num_epochs, lr, model, dataTrain, dataVal, opt_func)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


Epoch 1: train_loss: 0.920762300491333, val_loss: 0.8073042035102844, val_acc: 0.9453125
Epoch 2: train_loss: 0.7845194935798645, val_loss: 0.8051995635032654, val_acc: 0.9386160969734192
Epoch 3: train_loss: 0.7670024037361145, val_loss: 0.7919043302536011, val_acc: 0.9542410969734192
Epoch 4: train_loss: 0.7576763033866882, val_loss: 0.7860565781593323, val_acc: 0.9598214030265808
Epoch 5: train_loss: 0.7519763708114624, val_loss: 0.7847854495048523, val_acc: 0.9553571343421936
Epoch 6: train_loss: 0.7516723871231079, val_loss: 0.7830186486244202, val_acc: 0.9609375
Epoch 7: train_loss: 0.7507798671722412, val_loss: 0.7796981334686279, val_acc: 0.9619792103767395
Epoch 8: train_loss: 0.748507559299469, val_loss: 0.7801950573921204, val_acc: 0.9665178656578064
Epoch 9: train_loss: 0.748406708240509, val_loss: 0.7927884459495544, val_acc: 0.9519345164299011
Epoch 10: train_loss: 0.750078558921814, val_loss: 0.792548418045044, val_acc: 0.9508928656578064


In [63]:
evaluate(model, dataTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


{'val_loss': 0.7967091202735901, 'val_acc': 0.9445833563804626}

In [64]:
del(model)

# VGG


In [58]:
model3 = to_device(VGGNet(), device)
evaluate(model3, dataTest)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'val_loss': 1.3746545314788818, 'val_acc': 0.3095238208770752}

In [59]:
num_epochs = 15
opt_func = torch.optim.Adam
lr = 5.5e-5 

history = fit(num_epochs, lr, model3, dataTrain, dataVal, opt_func)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1: train_loss: 0.9677273631095886, val_loss: 0.8717265725135803, val_acc: 0.8646577596664429
Epoch 2: train_loss: 0.84929358959198, val_loss: 0.8476076722145081, val_acc: 0.8925595283508301
Epoch 3: train_loss: 0.8223062753677368, val_loss: 0.8415981531143188, val_acc: 0.8982142806053162
Epoch 4: train_loss: 0.8051403760910034, val_loss: 0.856124222278595, val_acc: 0.888244092464447
Epoch 5: train_loss: 0.7976633906364441, val_loss: 0.8913146257400513, val_acc: 0.8511160612106323
Epoch 6: train_loss: 0.7992236614227295, val_loss: 0.8433651328086853, val_acc: 0.896056592464447
Epoch 7: train_loss: 0.7842244505882263, val_loss: 0.7988829016685486, val_acc: 0.9418154954910278
Epoch 8: train_loss: 0.7712522745132446, val_loss: 0.8109492063522339, val_acc: 0.9317708611488342
Epoch 9: train_loss: 0.7673804759979248, val_loss: 0.8073222041130066, val_acc: 0.9340029954910278
Epoch 10: train_loss: 0.7665198445320129, val_loss: 0.8047615885734558, val_acc: 0.9407738447189331
Epoch 11: trai

In [60]:
evaluate(model3, dataTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'val_loss': 0.8236653208732605, 'val_acc': 0.917738139629364}

In [65]:
del(model3)

# MyLittleModel

In [ ]:
model2 = to_device(Net(), device)
evaluate(model2, dataTest)

In [ ]:
num_epochs = 25
opt_func = torch.optim.Adam
lr = 5.5e-5 

history = fit(num_epochs, lr, model2, dataTrain, dataVal, opt_func)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 1: train_loss: 1.5985661745071411, val_loss: 1.59907865524292, val_acc: 0.43465909361839294
Epoch 2: train_loss: 1.5879095792770386, val_loss: 1.5929450988769531, val_acc: 0.4517045319080353
Epoch 3: train_loss: 1.582120656967163, val_loss: 1.5874441862106323, val_acc: 0.4545454680919647
Epoch 4: train_loss: 1.576361060142517, val_loss: 1.5819647312164307, val_acc: 0.4630681872367859
Epoch 5: train_loss: 1.5712506771087646, val_loss: 1.5773649215698242, val_acc: 0.47159090638160706
Epoch 6: train_loss: 1.5662802457809448, val_loss: 1.5726343393325806, val_acc: 0.47159090638160706
Epoch 7: train_loss: 1.560564398765564, val_loss: 1.5653364658355713, val_acc: 0.4857954680919647
Epoch 8: train_loss: 1.553512454032898, val_loss: 1.569851279258728, val_acc: 0.4943181872367859
Epoch 9: train_loss: 1.546003818511963, val_loss: 1.5616538524627686, val_acc: 0.5056818127632141
Epoch 10: train_loss: 1.5410611629486084, val_loss: 1.5586732625961304, val_acc: 0.5
Epoch 11: train_loss: 1.53717

In [ ]:
evaluate(model2, dataTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:74: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


{'val_loss': 1.5115759372711182, 'val_acc': 0.524404764175415}

In [ ]:
list(dataTest)[0][1].argmax(1)

tensor([1, 1, 1, 1, 2, 2, 0, 0, 1, 1, 2, 2, 1, 1, 1, 1, 0, 0, 3, 3, 2, 2, 4, 4,
        0, 0, 1, 1, 2, 2, 3, 3, 2, 2, 1, 1, 1, 1, 2, 2, 0, 0, 5, 5, 3, 3, 2, 2,
        3, 3, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 1, 1, 2, 2,
        3, 3, 0, 0, 1, 1, 0, 0, 2, 2, 5, 5, 5, 5, 3, 3, 0, 0, 0, 0, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 1, 4, 4, 4, 4, 4, 4, 0, 0, 5, 5, 4, 4, 4, 4, 2, 2, 3, 3,
        1, 1, 1, 1, 3, 3, 3, 3], device='cuda:0')

In [ ]:
model(list(dataTest)[0][0]).argmax(1)

TypeError: ignored